In [43]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [44]:
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [45]:
X = tf.placeholder(tf.float32, [None, 28 * 28])
Y = tf.placeholder(tf.float32, [None, 10])
# 28 * 28픽셀로 이루어져 있다. 다시말해 784개의 특징으로 이루어져 있다고 할수 있다.
# 레이블은 0부터 9까지 이니 10개의 분류로 나누면 된다.

In [46]:
# 이미지를 하나씩 학습시키는것보다 여러개를 한꺼번에 학습시키는 쪽이 효과가 좋다. 그러나 그만큼 메모리와 높은 컴퓨팅 성능이 받침되어야 한다.
# 따라서 일반적으로 데이터를 적당한 크기로 잘라서 학습시키는데, 이것을 Mini Batch 라고 한다.
# X , Y코드에서 텐서의 첫번째 차원이 None 으로 되어있는것을 볼수 있다.
# 이 자리에는 한번에 학습시킬 MNIST 이미지의 개수를 지정하는 값이 들어간다. 즉, 배치 크기를 지정하는 자리이다.

keep_prob = tf.placeholder(tf.float32)
is_trainable = tf.placeholder(tf.bool)

# 첫번째 은닉층
W1 = tf.Variable(tf.random_normal([28 * 28, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
# L1 = tf.nn.dropout(L1, keep_prob)
L1 = tf.layers.batch_normalization(L1, training=is_trainable)

# 두번째 은닉층
W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))
# L2 = tf.nn.dropout(L2, keep_prob)
L2 = tf.layers.batch_normalization(L2, training=is_trainable)

# # 결과값
W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

In [47]:
# 가중치 많을 구성하여 보았다.
# tf.random_normal([256, 256], stddev=0.01) : 표준편차가 0.01인 정규분포를 가지는 임의의 값으로 뉴런(변수)를 초기화

# tf.nn.softmax_cross_entropy_with_logits 함수를 이용하여 각 이미지에 대한 손실값(실제값과 예측값의 차이)를 구하고,
# tf.reduce_mean함수를 사용해 미니배치의 평균 손실값을 구한다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [48]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [49]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
# 학습데이터의 총 개수인 mnist.train.num_examples를 배치크기로 나눠 미니배치가 총 몇개인지 저장

In [50]:
for epoch in range(30):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={
            X: batch_xs, 
            Y: batch_ys, 
#             keep_prob: 0.8,
            is_trainable: True
        })
        total_cost += cost_val
    print('Epoch: %04d' % (epoch + 1), 'Avg. cost = {:.3f}'.format(total_cost / total_batch) )
print('Optimization done!')

Epoch: 0001 Avg. cost = 0.196
Epoch: 0002 Avg. cost = 0.088
Epoch: 0003 Avg. cost = 0.063
Epoch: 0004 Avg. cost = 0.047
Epoch: 0005 Avg. cost = 0.040
Epoch: 0006 Avg. cost = 0.034
Epoch: 0007 Avg. cost = 0.027
Epoch: 0008 Avg. cost = 0.025
Epoch: 0009 Avg. cost = 0.024
Epoch: 0010 Avg. cost = 0.019
Epoch: 0011 Avg. cost = 0.016
Epoch: 0012 Avg. cost = 0.017
Epoch: 0013 Avg. cost = 0.016
Epoch: 0014 Avg. cost = 0.014
Epoch: 0015 Avg. cost = 0.013
Epoch: 0016 Avg. cost = 0.011
Epoch: 0017 Avg. cost = 0.012
Epoch: 0018 Avg. cost = 0.011
Epoch: 0019 Avg. cost = 0.010
Epoch: 0020 Avg. cost = 0.010
Epoch: 0021 Avg. cost = 0.009
Epoch: 0022 Avg. cost = 0.009
Epoch: 0023 Avg. cost = 0.007
Epoch: 0024 Avg. cost = 0.010
Epoch: 0025 Avg. cost = 0.008
Epoch: 0026 Avg. cost = 0.007
Epoch: 0027 Avg. cost = 0.007
Epoch: 0028 Avg. cost = 0.008
Epoch: 0029 Avg. cost = 0.006
Epoch: 0030 Avg. cost = 0.005
Optimization done!


In [51]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))

# 예측결과인 model의 값과 실제 레이블인 Y의 값을 비교

In [52]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [54]:
print('정확도:', sess.run(accuracy, feed_dict={
    X: mnist.test.images, 
    Y: mnist.test.labels, 
#     keep_prob: 1,
    is_trainable: True
}))

정확도: 0.9823
